In [2]:
%run './基本定义.ipynb'

# 1.开始训练模型

In [3]:

## 使用transE模型，wn18rr数据集

## 可修改model参数，训练rotate模型
## 可修改data_path参数，指定训练的数据集

base_args={
    'cuda': True,
    'evaluate_train': False,
    'double_entity_embedding': False, 
    'double_relation_embedding': False, 
    'negative_sample_size': 1024, 
    'hidden_dim': 200, 
    'gamma': 6.0, 
    'negative_adversarial_sampling': True, 
    'adversarial_temperature': 0.5, 
    'regularization': 0.0, 
    'test_batch_size': 8, 
    'uni_weight': False, 
    'learning_rate': 0.0005,
    'cpu_num': 10, 
    'comments': '\n', 
}

args={
    'do_train': True, 
    'do_test': True, 
    'data_path': 'data/wn18rr', 
    'model': 'TransE', 
    'batch_size': 512, 
    'max_steps': 40000, ## 经验数据 RotatE FB15k-237 30000
                        ## RotatE wn18rr 30000
                        ## TransE FB15k-237 20000
                        ## TransE wn18rr 40000
    'warm_up_steps': 20000, 
    'no_save': True, 
    'save_path': './models/TransE_wn18rr_baseline', 
    'save_checkpoint_steps': 10000, 
    'valid_steps': 10000, 
    'log_steps': 2000, 
    'classify_steps': 5000, 
    'test_log_steps': 1000, 
    'nentity': 40943, 
    'nrelation': 11,
    'fake':None,
    'do_valid':False
}
args = {**base_args,**args}
log_filename = "train"
set_logger(args, filename=log_filename)
input_data = get_input_data(args)
trainer = BaseTrainer.get_trainer(input_data, args)
kge_model = trainer.kge_model
logging.info('Start Training...')
logging.info(f"args is {args}")
logging.info('learning_rate = %f' % trainer.lr)
for step in range(args['max_steps']):
    trainer.basicTrainStep(step)
trainer.save_model()

2023-12-19 17:28:16,800 INFO     

2023-12-19 17:28:16,802 INFO     Model: TransE
2023-12-19 17:28:16,803 INFO     Data Path: data/wn18rr
2023-12-19 17:28:16,804 INFO     #entity: 40943
2023-12-19 17:28:16,805 INFO     #relation: 11
2023-12-19 17:28:16,805 INFO     #train: 86835	#valid: 3034	#test: 3134
2023-12-19 17:28:23,817 INFO     Model Parameter Configuration:
2023-12-19 17:28:23,818 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 17:28:23,819 INFO     Parameter embedding_range: torch.Size([1]), require_grad = False
2023-12-19 17:28:23,820 INFO     Parameter entity_embedding: torch.Size([40943, 200]), require_grad = True
2023-12-19 17:28:23,820 INFO     Parameter relation_embedding: torch.Size([11, 200]), require_grad = True
2023-12-19 17:28:23,822 INFO     Start Training...
2023-12-19 17:28:23,823 INFO     args is {'cuda': True, 'evaluate_train': False, 'double_entity_embedding': False, 'double_relation_embedding': False, 'negative_sample_size': 1024, 

# 2. 验证集

In [4]:
logging.info('Evaluating on Valid Dataset...')
metrics = kge_model.test_step(kge_model, input_data.valid_triples, input_data.all_true_triples, args)
log_metrics('Valid', args['max_steps'], metrics)

2023-12-19 17:48:02,680 INFO     Evaluating on Valid Dataset...
2023-12-19 17:48:03,622 INFO     Evaluating the model... (0/760)
2023-12-19 17:48:24,820 INFO     len of triple2mode2ranking: 3034
2023-12-19 17:48:24,839 INFO     Valid MRR at step 40000: 0.224202
2023-12-19 17:48:24,840 INFO     Valid MR at step 40000: 3315.136948
2023-12-19 17:48:24,841 INFO     Valid HITS@1 at step 40000: 0.015985
2023-12-19 17:48:24,842 INFO     Valid HITS@3 at step 40000: 0.400626
2023-12-19 17:48:24,842 INFO     Valid HITS@10 at step 40000: 0.527851
2023-12-19 17:48:24,843 INFO     Valid 1_MR at step 40000: 1.000000
2023-12-19 17:48:24,844 INFO     Valid 1_MRR at step 40000: 1.000000
2023-12-19 17:48:24,844 INFO     Valid 1_NUM at step 40000: 60.000000
2023-12-19 17:48:24,845 INFO     Valid 10_MR at step 40000: 2.799632
2023-12-19 17:48:24,846 INFO     Valid 10_MRR at step 40000: 0.425411
2023-12-19 17:48:24,846 INFO     Valid 10_NUM at step 40000: 2720.000000
2023-12-19 17:48:24,849 INFO     Valid 

# 3. 测试集

In [5]:
logging.info('Evaluating on Test Dataset...')
metrics = kge_model.test_step(kge_model, input_data.test_triples, input_data.all_true_triples, args)
log_metrics('Test', args['max_steps'], metrics)

2023-12-19 17:48:34,628 INFO     Evaluating on Test Dataset...
2023-12-19 17:48:35,381 INFO     Evaluating the model... (0/784)
2023-12-19 17:48:57,039 INFO     len of triple2mode2ranking: 3134
2023-12-19 17:48:57,057 INFO     Test MRR at step 40000: 0.225509
2023-12-19 17:48:57,058 INFO     Test MR at step 40000: 3286.567645
2023-12-19 17:48:57,059 INFO     Test HITS@1 at step 40000: 0.016433
2023-12-19 17:48:57,060 INFO     Test HITS@3 at step 40000: 0.402521
2023-12-19 17:48:57,061 INFO     Test HITS@10 at step 40000: 0.529675
2023-12-19 17:48:57,062 INFO     Test 1_MR at step 40000: 1.000000
2023-12-19 17:48:57,063 INFO     Test 1_MRR at step 40000: 1.000000
2023-12-19 17:48:57,063 INFO     Test 1_NUM at step 40000: 62.000000
2023-12-19 17:48:57,064 INFO     Test 10_MR at step 40000: 2.780384
2023-12-19 17:48:57,065 INFO     Test 10_MRR at step 40000: 0.424704
2023-12-19 17:48:57,066 INFO     Test 10_NUM at step 40000: 2814.000000
2023-12-19 17:48:57,066 INFO     Test 100_MR at ste